# Using the ESCWCC API in Python

Import required packages

In [1]:
import requests
import json
import pandas
pandas.options.mode.chained_assignment = None

## Establish API GET request URL

In [2]:
base_url = "http://209.97.132.245:8000/ESCWCC?"

Set the longitude (lon) and latitude (lat) in EPSG:3857.

In [41]:
lon = -141102.7542
lat = 6830259.9735

# lon = -184144
# lat = 7171860

Establish the site modifier options.

* drainage:int
    * 0 = None.
    * 1 = Drainage installed.
* brash:int
    * 0 = None (new planting).
    * 1 = Fresh brash (<18 months).
    * 2 = Brash (>18 months).
* fertiliser:int
    * 0 = None.
    * 1 = Fertiliser applied.
* exposure:int
    * 0 = None.
    * 1 = Exposed ridge/Coastal zone.
    * 2 = Local shelter/Less exposed/Underplanting.

In [4]:
drainage = 0
brash = 0
fertiliser = 0
exposure = 0

Establish the climate scenario/s (climate), species group filter (filter), and ESC version.

* climate:int
    * 0 = Baseline.
    * 1 = 2050.
    * 2 = 2080.
    * 3 = Baseline, 2050, and 2080.
* filter:int
    * 0 = No filter
    * 1 = Conifers only
    * 2 = Broadleaves only
    * 3 = Natives only
    * 4 = Non-natives only
* escVersion:float
    * 4.33 = 4.33

In [19]:
climate = 3
filter = 0
escVersion = 4.33

Establish the Woodland Carbon Code (WCC) version (wccVersion), the planting spacing (wccSpacing), management regime (thinning or no thinning) (wccManagement), period (wccPeriod), area of the site to be established (wccEstArea) in hectares (ha), the length of roads to be constructed (wccRoadLength) in km, and the area, if any, of arable land with mineral soil to be planted (wccSoilArea) in ha.
* wccVersion:float
    * 2.4 = v2.4 (March 2021)
* wccSpacing:list
    * 0 = 'all', eqiuvalent to {0: [1.2, 2.5, 3.0, 1.4, 1.7, 1.8, 1.5, 2.0]}
* wccManagement:list
    * 0 = 'all', eqiuvalent to {0: ["NO_thin", "Thinned"]}
* wccPeriod:list
    * 0 = 'all', eqiuvalent to {0: ["0", "0-5", "5-10", "10-15", "15-20", "20-25", "25-30", "30-35", "35-40", "40-45", "45-50", 
                                    "50-55", "55-60", "60-65", "65-70", "70-75", "75-80", "80-85", "85-90", "90-95", "95-100", 
                                    "100-105", "105-110", "110-115", "115-120", "120-125", "125-130", "130-135", "135-140", 
                                    "140-145", "145-150", "150-155", "155-160", "160-165", "165-170", "170-175", "175-180", 
                                    "180-185", "185-190", "190-195", "195-200"]}
* wccEstArea:float
    * 0 by default, any positive number.
* wccRoadLength:float
    * 0 by default, any positive number.
* wccSoilArea:float
    * 0 by default, any positive number.

In [20]:
wccVersion = 0 #2.4
wccSpacing = 0
wccManagement = 0
wccPeriod = 0
wccEstArea = 12
wccRoadLength = 1
wccSoilArea = 2

Establish the manual Soil Moisture Regime (SMR) and Soil Nutrient Regime (SNR) modifier values.
Enter None, or omit from the request dictionary (`req_dict`) below if you do not wish to manually alter the values.

* smr:float, any number between 0 and 8. Indicative categorical values are as follows:
    * 1 = Very Wet
    * 2 = Wet 
    * 3 = Moist
    * 4 = Moist
    * 5 = Fresh
    * 6 = Slightly Dry
    * 7 = Moderately Dry
    * 8 = Very Dry
* snr:float, any number between 0 and 6. Indicative categorical values are as follows:
    * 0 = Very Poor (VP1)
    * 0.5 = Very Poor (VP2)
    * 1 = Very Poor (VP3)
    * 1.5 = Very Poor - Poor
    * 2 = Poor
    * 3 = Medium
    * 4 = Rich
    * 5 = Very Rich
    * 6 = Carbonate

In [21]:
smr = None
snr = None

Compose the request dictionary. Optionally, you can accept the default values for all parameters apart from lon and lat by emitting the parameters from the request dictionary (as shown in the commented out code below).

In [42]:
# req_dict = {"lon": lon, "lat": lat}

req_dict = {"lon": lon, "lat": lat, "drainage": drainage, "brash": brash, "fertiliser": fertiliser, "exposure": exposure, 
            "climate": climate, "filter": filter, "escVersion": escVersion, "wccVersion": wccVersion, "smr": smr, "snr": snr,
            "wccSpacing": wccSpacing, "wccManagement": wccManagement, "wccPeriod": wccPeriod,
            "wccEstArea": wccEstArea, "wccRoadLength": wccRoadLength, "wccSoilArea": wccSoilArea}

req_dict


{'lon': -141102.7542,
 'lat': 6830259.9735,
 'drainage': 0,
 'brash': 0,
 'fertiliser': 0,
 'exposure': 0,
 'climate': 3,
 'filter': 0,
 'escVersion': 4.33,
 'wccVersion': 0,
 'smr': None,
 'snr': None,
 'wccSpacing': 0,
 'wccManagement': 0,
 'wccPeriod': 0,
 'wccEstArea': 12,
 'wccRoadLength': 1,
 'wccSoilArea': 2}

## Make the API call

Call the ESC API using the base url and request dictionary

In [43]:
ESCresponse = requests.get(url = base_url, params = req_dict)
ESCresponse.json

<bound method Response.json of <Response [200]>>

Establish a function to deserialize the json response into a nested dictionary.

In [24]:
def deserializeESCTreeResults(nestedJson:str) -> dict:
    
    loadedJson = json.loads(nestedJson)

    for k,v in loadedJson.items():
        loadedJson[k] = json.loads(v)
        for t,c in loadedJson[k].items():
            loadedJson[k][t] = pandas.DataFrame.from_dict(data = json.loads(c))
            
    return loadedJson

Call the `deserializeResults` function on the ESC API output.

In [44]:
ESCresponse_dict = deserializeESCTreeResults(ESCresponse.json())
ESCresponse_dict.keys()

dict_keys(['Baseline', '2050', '2080'])

## Explore Results

The object produced (`ESCresponse_dict`) is a nested dictionary with the following structure:

```
ESCresponse_dict (dict)
├── Metadata (dict)
│   ├── ESCMetadata (DataFrame)
│   ├── WCCMetadata (DataFrame)
├── Baseline (dict)
│   ├── ESCDoseVals (DataFrame)
│   ├── ESCResponseVals (DataFrame)
│   ├── ESCSuitMetrics (DataFrame)
│   ├── WCCTreeValues (DataFrame)
│   ├── WCCSoilLoss (DataFrame)
│   ├── WCCSoilSeq (DataFrame)
│   ├── WCCEstablishment (DataFrame)
├── 2050 (dict)
│   ├── ESCDoseVals (DataFrame)
│   ├── ESCResponseVals (DataFrame)
│   ├── ESCSuitMetrics (DataFrame)
│   ├── WCCTreeValues (DataFrame)
│   ├── WCCSoilLoss (DataFrame)
│   ├── WCCSoilSeq (DataFrame)
│   ├── WCCEstablishment (DataFrame)
├── 2080 (dict)
│   ├── ESCDoseVals (DataFrame)
│   ├── ESCResponseVals (DataFrame)
│   ├── ESCSuitMetrics (DataFrame)
│   ├── WCCTreeValues (DataFrame)
│   ├── WCCSoilLoss (DataFrame)
│   ├── WCCSoilSeq (DataFrame)
│   ├── WCCEstablishment (DataFrame)
```

In [26]:
ESCresponse_dict.get("Baseline").keys()

dict_keys(['ESCDoseVals', 'ESCResponseVals', 'ESCSuitMetrics', 'WCCTreeValues', 'WCCEstablishment', 'WCCSoilLoss', 'WCCSoilSeq'])

Inspect the metadata for the ESC and WCC values.

In [ ]:
#ESCresponse_dict.get("Metadata").get("ESCMetadata")

In [ ]:
#ESCresponse_dict.get("Metadata").get("WCCMetadata")

Access the ESC & WCC data frames from the API response for a given climate scenario using the following code.

The `ESCDoseVals` data frame contains the environmental dose variables obtained from the modelled data and/or manually entered, which are then modified by the aforementioned API query parameters `drainage`, `brash`, `fertiliser`, and `exposure`.
For more information on the underlying method see the [Forest Research Bulletin 124](https://www.forestresearch.gov.uk/publications/archive-an-ecological-site-classification-for-forestry-in-great-britain/) and the [Forest Research Technical Paper 20](https://www.forestresearch.gov.uk/publications/archive-an-ecological-site-classification-for-forestry-in-great-britain-with-special-reference-to-grampian-scotland/).
Column definitions:
* SMR - soil moisture regime
* SNR - soil nutrient regime
* AT - accumulated temperature
* CT - continentality
* DAMS - direct aspect method of scoring
* MD - moisture deficit
* Item - The stage of modification of the environmental dose variable values.

In [30]:
baseline_ESCDoseVals = ESCresponse_dict.get("Baseline").get("ESCDoseVals")
baseline_ESCDoseVals

,SMR,SNR,AT,CT,DAMS,MD,Item
0,3.0,3.0,1422.723145,9.027328,12.510952,145.714996,Values - Base
1,NaN,NaN,NaN,NaN,NaN,NaN,Values - Manual
2,3.0,3.0,1422.723145,9.027328,12.510952,145.714996,Values - PreMod
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Modifier - Drainage
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Modifier - Brash
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Modifier - Fertiliser
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,Modifier - Exposure
7,3.0,3.0,1422.723145,9.027328,12.510952,145.714996,Values - Final


The `ESCResponseVals` data frame contains data on the response of each tree species to each environmental dose metric.
Column definitions:
* speciesCode - The Forest Research species code for the species.
* metric - The environmental dose variable (see above).
* response - The response of the tree species to the environmental dose variable. A unitless value between 0 and 1.

In [31]:
baseline_ESCResponseVals = ESCresponse_dict.get("Baseline").get("ESCResponseVals")
baseline_ESCResponseVals.head()

,speciesCode,metric,response
0,AH,AT,1.000000
1,AH,CT,1.000000
2,AH,DAMS,0.944794
3,AH,MD,1.000000
4,AH,SMR,0.755714


The `ESCSuitMetrics` data frame contains a range of summary data relating to the suitability of each tree species based on the `ESCResponseVals` data.
Column definitions:
* speciesCode - The Forest Research species code for the species.
* speciesScientific - The scientific name of the species.
* speciesName - The common name of the species.
* maxYC - The maximum yield class of that species
* threat - Any major threats to that tree species which should be taken into account when considering planting that species.
* limFactorInclAT - The limiting factor (metric with the lowest response) including AT.
* limFactorInclATResponse - The response value for limiting factor (metric with the lowest response) including AT.
* limFactorExclAT - The limiting factor (metric with the lowest response) excluding AT.
* limFactorExclATResponse - The response value for limiting factor (metric with the lowest response) excluding AT.
* TimberSuitability - The suitability of that tree species for planting as a timber crop.
* EcologicalSuitability - The suitability of that tree species for planting.
* estimatedYC - The estimated yield class.


In [63]:
baseline_ESCSuitMetrics = ESCresponse_dict.get("Baseline").get("ESCSuitMetrics")
# print(baseline_ESCSuitMetrics.to_string())
baseline_ESCSuitMetrics_nozero = baseline_ESCSuitMetrics[baseline_ESCSuitMetrics["estimatedYC"] != 0].reset_index(drop = True)
print(baseline_ESCSuitMetrics_nozero.speciesScientific.to_string())

0             Fraxinus excelsior
1                Populus tremula
2                Fagus sylvatica
3                  Populus nigra
4                Alnus glutinosa
5                    Pinus nigra
6              Eucalyptus gunnii
7                  Larix decidua
8              Eucalyptus nitens
9                     Abies alba
10                  Alnus incana
11               Carpinus betula
12               Larix euolepsis
13               Ilex aquifolium
14              Picea sitchensis
15          Cryptomeria japonica
16               Larix kaempferi
17                 Juglans regia
18      Chamaecyparis lawsoniana
19    Cupressocyparis x leyandii
20                Pinus contorta
21                Pinus pinaster
22                   Pinus peuce
23            Abies nordmanianna
24              Acer platanoides
25                   Picea abies
26                 Picea omorika
27              Picea orientalis
28              Betula pubescens
29                 Quercus robur
30        

The `WCCTreeValues` data frame contains data from the woodland carbon code biomass carbon lookup tables (BCLT) and clearfell max sequence values for the yield class nearest to the ESC estimated yield class.
Column definitions:
* speciesCode - The Forest Research species code for the species.
* wccCode - The WCC species code mapped to the ESC species, from which the WCC data is retrieved.
* speciesScientific - The scientific name of the species.
* speciesName - The common name of the species.
* estimatedYC - The estimated yield class.
* YC - The yield class associated with the biomass carbon lookup tables (BCLT) and clearfell max sequence values WCC data. Retrieved from column D in the BCLT tables.
* spacing - The planting spacing. Retrieved from column C in the BCLT tables.
* management - The management regime for the planted area, thinned or un-thinned. Retrieved from column E in the BCLT tables.
* period - The period of years associated with the wcc data. Retrieved from column F in the BCLT tables.
* standingCarbonYr - The carbon contained within the above-ground biomass of the trees planted on site. Retrieved from column G in the BCLT tables. Units of tCO2e/ha/yr.
* debrisCarbonYr - The carbon contained within the debris on site. Retrieved from column H in the BCLT tables. Units of tCO2e/ha/yr.
* totalCarbonYr - The total carbon contained on site, equal to the sum of standingCarbonYr and debrisCarbonYr. Retrieved from column I in the BCLT tables. Units of tCO2e/ha/yr.
* cumCarbon5Yr - The cumulative carbon sequestered on site over the period stipulated in the period column. Retrieved from column J in the BCLT tables. Units of tCO2e/ha/period.
* cumBiomassCarbon - The cumulative carbon sequestered on site over up until the period stipulated in the period column. Retrieved from column K in the BCLT tables. Units of tCO2e/ha.
* cumEmissionsMgmt - The cumulative emissions from ongiong management on site. Retrieved from column L in the BCLT tables. Units of tCO2e/ha.
* cumTotalCarbon - The total cumulative emissions on site. Retrieved from column M in the BCLT tables. Units of tCO2e/ha.
* remCarbonYr - The quantity of carbon removed from the site yearly. Retrieved from column N in the BCLT tables. Units of tCO2e/ha/yr.
* periodEnd - The rotation length stipulated in columns F to AK of the "Clearfell_Max_Seq_Values" sheet of the excel tool. Mapped to the last year of the period present in the period column.
* clearfellCapCarbon - The Clearfell Cap Value associated with the periodEnd, wccCode, spacing, YC, and management. Units of tCO2e.


In [62]:
baseline_WCCTreeValues = ESCresponse_dict.get("Baseline").get("WCCTreeValues")
# baseline_WCCTreeValues.head()
print(baseline_WCCTreeValues.to_string())

     speciesCode wccCode              speciesScientific                    speciesName  estimatedYC    YC  spacing management   period  standingCarbonYr  debrisCarbonYr  totalCarbonYr  cumCarbon5Yr  cumBiomassCarbon  cumEmissionsMgmt  cumTotalCarbon  remCarbonYr  periodEnd  clearfellCapCarbon
0             AH     SAB             Fraxinus excelsior                            Ash     9.068572  10.0      1.5    NO_thin      0-5              4.07            0.52           4.59          23.0              23.0              0.00            23.0         0.00        5.0                 6.5
1             AH     SAB             Fraxinus excelsior                            Ash     9.068572  10.0      1.5    NO_thin     5-10             12.42            0.65          13.06          65.3              88.3              0.00            88.3         0.00       10.0                13.0
2             AH     SAB             Fraxinus excelsior                            Ash     9.068572  10.0      1.5    

The `WCCEstablishment` data frame contains the establishment emission values calculated using the same methodology as in the "StandardProjectCarbonCalculator" sheet of the excel tool.
Column definitions:
* seedlingSpacing
* variable - the establishment emission source, one of:
    * seedling
    * groundPrepFuel
    * treeShelters
    * fencing
    * herbicide
    * totalEmissions
* totalEmissions- the carbon emissions associated with that seedlingSpacing and variable. Units of tCO2e.

In [ ]:
baseline_WCCEstablishment = ESCresponse_dict.get("Baseline").get("WCCEstablishment")
baseline_WCCEstablishment.head()

,seedlingSpacing,variable,totalEmissions
0,1.2,seedling,-12.60
1,1.4,seedling,-9.24
2,1.5,seedling,-8.04
3,1.7,seedling,-6.24
4,2.0,seedling,-4.56


The `WCCSoilLoss` data frame contains data on the soil carbon emissions during the year of establishment (Year 1) for each country, percentage topsoil carbon to be subtracted and previous land use.
Column definitions:
* Country - The country in which the site is located.
* topsoilCarbonSubtract - the percentage of topsoil carbon to subtract .
* Seminatural - the soil carbon emissions associated with the previous land-use "Semi-natural", units in tCO2e/ha.
* Pasture - the soil carbon emissions associated with the previous land-use "Pasture, units in tCO2e/ha.
* Arable - the soil carbon emissions associated with the previous land-use "Arable", units in tCO2e/ha.


In [ ]:
baseline_WCCSoilLoss = ESCresponse_dict.get("Baseline").get("WCCSoilLoss")
baseline_WCCSoilLoss.head()

,Country,topsoilCarbonSubtract,Seminatural,Pasture,Arable
0,England,0,0.0,0.000000,0.000000
1,Scotland,0,0.0,0.000000,0.000000
2,Wales,0,0.0,0.000000,0.000000
3,Northern Ireland,0,0.0,0.000000,0.000000
4,England,2,-8.8,-5.866667,-5.133333


The `WCCSoilSeq` data frame contains data on the cumulative soil carbon sequestration for mineral soils which where the former land use was arable agriculture. 
Column definitions:
* period - The period of years associated with the wcc data. Retrieved from column Y in the "StandardProjectCarbonCalculator" sheet of the excel tool.
* cumSoilSeq - Calculated by multiplying column AY in the "StandardProjectCarbonCalculator" by the site area. Units in tCO2e.

In [ ]:
baseline_WCCSoilSeq = ESCresponse_dict.get("Baseline").get("WCCSoilSeq")
baseline_WCCSoilSeq.head()

,period,cumSoilSeq
0,0-5,5.5
1,5-10,11.0
2,10-15,16.5
3,15-20,22.0
4,20-25,27.5


## Calculate Claimable Carbon Sequestration 

To calculate the average total claimable sequestration by year in tCO2e/ha perform the following calculations using the API outputs.

Columns suffixed with (<[A-Z]>) correspond to the columns CB to CM of Version 2.4 (March 2021) of the WCC excel tool.

First, retrieve the WCC Biomass Carbon Lookup Table (BCLT) and Clearfell Max Seq Values (CMSV) values for selected species, e.g. Rowan.

In [ ]:
baseline_WCCTreeValues_ROW = baseline_WCCTreeValues[baseline_WCCTreeValues["speciesCode"] == "ROW"]
baseline_WCCTreeValues_ROW.head()

,speciesCode,wccCode,speciesScientific,speciesName,estimatedYC,YC,spacing,management,period,standingCarbonYr,debrisCarbonYr,totalCarbonYr,cumCarbon5Yr,cumBiomassCarbon,cumEmissionsMgmt,cumTotalCarbon,remCarbonYr,periodEnd,clearfellCapCarbon
1440,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,NO_thin,0-5,NaN,NaN,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN
1441,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,NO_thin,5-10,NaN,NaN,NaN,NaN,NaN,NaN,9.6,NaN,NaN,NaN
1442,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,NO_thin,10-15,NaN,NaN,NaN,NaN,NaN,NaN,28.9,NaN,NaN,NaN
1443,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,NO_thin,15-20,NaN,NaN,NaN,NaN,NaN,NaN,76.4,NaN,NaN,NaN
1444,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,NO_thin,20-25,NaN,NaN,NaN,NaN,NaN,NaN,120.6,NaN,NaN,NaN


Select the BCLT and CMSV values for a particular spacing and management regime, e.g. 1.5m and Thinned.

In [ ]:
baseline_WCCTreeValues_ROW_1p5_Thinned = baseline_WCCTreeValues_ROW[(baseline_WCCTreeValues_ROW.spacing == 1.5) & (baseline_WCCTreeValues_ROW.management == "Thinned")]
baseline_WCCTreeValues_ROW_1p5_Thinned.head()

,speciesCode,wccCode,speciesScientific,speciesName,estimatedYC,YC,spacing,management,period,standingCarbonYr,debrisCarbonYr,totalCarbonYr,cumCarbon5Yr,cumBiomassCarbon,cumEmissionsMgmt,cumTotalCarbon,remCarbonYr,periodEnd,clearfellCapCarbon
1480,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,Thinned,0-5,NaN,NaN,NaN,NaN,NaN,NaN,2.7,NaN,NaN,NaN
1481,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,Thinned,5-10,NaN,NaN,NaN,NaN,NaN,NaN,9.6,NaN,NaN,NaN
1482,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,Thinned,10-15,NaN,NaN,NaN,NaN,NaN,NaN,28.9,NaN,NaN,NaN
1483,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,Thinned,15-20,NaN,NaN,NaN,NaN,NaN,NaN,70.6,NaN,NaN,NaN
1484,ROW,SAB,Sorbus aucuparia,Rowan,2.4,2.0,1.5,Thinned,20-25,NaN,NaN,NaN,NaN,NaN,NaN,91.1,NaN,NaN,NaN


Define a species composition for the site and join to a trimmed WCCTreeValues dataframe.

In [ ]:
species_composition = pandas.DataFrame.from_dict(data = {"speciesCode": "ROW", "proportion": [1]})
species_composition

,speciesCode,proportion
0,ROW,1


In [ ]:
df = baseline_WCCTreeValues_ROW_1p5_Thinned

df = df[["speciesCode", "period", "cumTotalCarbon"]]
df = df.merge(species_composition, on = ["speciesCode"])
df.head()

,speciesCode,period,cumTotalCarbon,proportion
0,ROW,0-5,2.7,1
1,ROW,5-10,9.6,1
2,ROW,10-15,28.9,1
3,ROW,15-20,70.6,1
4,ROW,20-25,91.1,1


Multiply the 'Cumulative total sequestration" data (cumTotalCarbon) by the species proportion and estimated area.

In [ ]:
df["cumTotalCarbonSite (A)"] = df["cumTotalCarbon"] * df["proportion"] * wccEstArea
df.head()

,speciesCode,period,cumTotalCarbon,proportion,cumTotalCarbonSite (A)
0,ROW,0-5,2.7,1,32.4
1,ROW,5-10,9.6,1,115.2
2,ROW,10-15,28.9,1,346.8
3,ROW,15-20,70.6,1,847.2
4,ROW,20-25,91.1,1,1093.2


Sum across all species

In [ ]:
df = df.drop(columns = ["speciesCode", "cumTotalCarbon", "proportion"])
df = df.set_index("period")
df.sum(axis = 0).to_frame()
df = df.reset_index()
df.head()

,period,cumTotalCarbonSite (A)
0,0-5,32.4
1,5-10,115.2
2,10-15,346.8
3,15-20,847.2
4,20-25,1093.2


Add a 20% buffer by multiplying the Cumulative total sequestration for the site by 0.8

In [ ]:
df["cumTotalCarbonSiteReduced (B)"] = df["cumTotalCarbonSite (A)"] * 0.8
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B)
0,0-5,32.4,25.92
1,5-10,115.2,92.16
2,10-15,346.8,277.44
3,15-20,847.2,677.76
4,20-25,1093.2,874.56


Retrieve the total establishment emissions for the selected spacing and add to the dataframe.

In [ ]:
totalEstablishmentC = baseline_WCCEstablishment[(baseline_WCCEstablishment.seedlingSpacing == 1.5) & (baseline_WCCEstablishment.variable == "total")]
totalEstablishmentC = totalEstablishmentC.drop(columns = ["seedlingSpacing", "variable"]).reset_index(drop = True)
totalEstablishmentC = totalEstablishmentC.rename(columns = {"totalEmissions": "establishmentCarbonLoss (C)"})
df["establishmentCarbonLoss (C)"] = totalEstablishmentC._get_value(0, 0, takeable = True)
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B),establishmentCarbonLoss (C)
0,0-5,32.4,25.92,-81.412
1,5-10,115.2,92.16,-81.412
2,10-15,346.8,277.44,-81.412
3,15-20,847.2,677.76,-81.412
4,20-25,1093.2,874.56,-81.412


Calculate soil carbon flux by summing soil loss and soil sequestration data .
To do this first identify the % topsoil carbon (0-30cm) loss associated with the method of site preparation and soil type.

In [ ]:
establishmentMethods = pandas.DataFrame.from_dict(data = {"MethodSitePrep": ["Negligible Disturbance", "Low Disturbance", "Medium Disturbance", "High Disturbance", "Very High Disturbance"],
                                                          "Organomineral": [0, 5, 10, 20, 40],
                                                          "Mineral": [0, 0, 2, 5, 10]})

establishmentMethods = establishmentMethods.set_index("MethodSitePrep")
percSoilLoss = establishmentMethods._get_value("Medium Disturbance", "Mineral")
percSoilLoss

2

Retrieve the soil carbon lost for the selected country, previous land use, and % topsoil carbon (0-30cm) loss identified above.

In [ ]:
soilCLoss = baseline_WCCSoilLoss[baseline_WCCSoilLoss["topsoilCarbonSubtract"] == percSoilLoss]
soilCLoss = soilCLoss.set_index("Country")
soilCLoss = soilCLoss._get_value("England", "Arable")
soilCLoss

-5.133333333

In [ ]:
soilCFlux = baseline_WCCSoilSeq
soilCFlux["cumSoilLoss"] = soilCLoss
soilCFlux["cumSoilCarbonFlux (D)"] = soilCFlux["cumSoilLoss"] + soilCFlux["cumSoilSeq"]
soilCFlux = soilCFlux.drop(columns = ["cumSoilSeq", "cumSoilLoss"])
soilCFlux.head()

,period,cumSoilCarbonFlux (D)
0,0-5,0.366667
1,5-10,5.866667
2,10-15,11.366667
3,15-20,16.866667
4,20-25,22.366667


In [ ]:
df = df.merge(soilCFlux, on = ["period"])
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B),establishmentCarbonLoss (C),cumSoilCarbonFlux (D)
0,0-5,32.4,25.92,-81.412,0.366667
1,5-10,115.2,92.16,-81.412,5.866667
2,10-15,346.8,277.44,-81.412,11.366667
3,15-20,847.2,677.76,-81.412,16.866667
4,20-25,1093.2,874.56,-81.412,22.366667


Calculate the total cumulative carbon sequestration

In [ ]:
df["totalProjectCarbonSeq (E)"] = df["cumTotalCarbonSiteReduced (B)"] + df["establishmentCarbonLoss (C)"] + df["cumSoilCarbonFlux (D)"]
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B),establishmentCarbonLoss (C),cumSoilCarbonFlux (D),totalProjectCarbonSeq (E)
0,0-5,32.4,25.92,-81.412,0.366667,-55.125333
1,5-10,115.2,92.16,-81.412,5.866667,16.614667
2,10-15,346.8,277.44,-81.412,11.366667,207.394667
3,15-20,847.2,677.76,-81.412,16.866667,613.214667
4,20-25,1093.2,874.56,-81.412,22.366667,815.514667


Optionally, add Baseline and Leakage data then calculate the adjusted, net total project carbon sequestration

In [ ]:
df["Baseline (F)"] = 0
df["Leakage (G)"] = 0
df["netTotalProjectCarbonSeq (H)"] = df["totalProjectCarbonSeq (E)"]+ df["Baseline (F)"] + df["Leakage (G)"]
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B),establishmentCarbonLoss (C),cumSoilCarbonFlux (D),totalProjectCarbonSeq (E),Baseline (F),Leakage (G),netTotalProjectCarbonSeq (H)
0,0-5,32.4,25.92,-81.412,0.366667,-55.125333,0,0,-55.125333
1,5-10,115.2,92.16,-81.412,5.866667,16.614667,0,0,16.614667
2,10-15,346.8,277.44,-81.412,11.366667,207.394667,0,0,207.394667
3,15-20,847.2,677.76,-81.412,16.866667,613.214667,0,0,613.214667
4,20-25,1093.2,874.56,-81.412,22.366667,815.514667,0,0,815.514667


Calculate the claimable carbon sequestration by adding a final 20% buffer

In [ ]:
df["claimableCarbonSeq (J)"] = df["netTotalProjectCarbonSeq (H)"] * 0.8
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B),establishmentCarbonLoss (C),cumSoilCarbonFlux (D),totalProjectCarbonSeq (E),Baseline (F),Leakage (G),netTotalProjectCarbonSeq (H),claimableCarbonSeq (J)
0,0-5,32.4,25.92,-81.412,0.366667,-55.125333,0,0,-55.125333,-44.100267
1,5-10,115.2,92.16,-81.412,5.866667,16.614667,0,0,16.614667,13.291733
2,10-15,346.8,277.44,-81.412,11.366667,207.394667,0,0,207.394667,165.915733
3,15-20,847.2,677.76,-81.412,16.866667,613.214667,0,0,613.214667,490.571733
4,20-25,1093.2,874.56,-81.412,22.366667,815.514667,0,0,815.514667,652.411733


Calculate the mean claimable carbon sequestration per hectare, in tCO2e per ha per year

In [389]:
df["meanClaimableCarbonSeqPerHa (K)"] = df["claimableCarbonSeq (J)"] / wccEstArea
df.head()

,period,cumTotalCarbonSite (A),cumTotalCarbonSiteReduced (B),establishmentCarbonLoss (C),cumSoilCarbonFlux (D),totalProjectCarbonSeq (E),Baseline (F),Leakage (G),netTotalProjectCarbonSeq (H),claimableCarbonSeq (J),meanClaimableCarbonSeqPerHa (K)
0,0-5,32.4,25.92,-81.412,0.366667,-55.125333,0,0,-55.125333,-44.100267,-3.675022
1,5-10,115.2,92.16,-81.412,5.866667,16.614667,0,0,16.614667,13.291733,1.107644
2,10-15,346.8,277.44,-81.412,11.366667,207.394667,0,0,207.394667,165.915733,13.826311
3,15-20,847.2,677.76,-81.412,16.866667,613.214667,0,0,613.214667,490.571733,40.880978
4,20-25,1093.2,874.56,-81.412,22.366667,815.514667,0,0,815.514667,652.411733,54.367644
